In [ ]:
#some imports
from bs4 import BeautifulSoup
from dateutil import parser
from difflib import SequenceMatcher
import re,os
import glob

In [ ]:
#FOR TOPIC MODELLING
#https://de.dariah.eu/tatom/topic_model_python.html
import numpy as np  # a conventional alias

import sklearn.feature_extraction.text as text
from sklearn import decomposition

In [ ]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
def language_detection_with_pyenchant(string_to_read):
    #https://stackoverflow.com/questions/3788870/how-to-check-if-a-word-is-an-english-word-with-python
    import enchant
    lg_ang=0
    us = enchant.Dict("en_US")
    #print "US LOADED"
    fr = enchant.Dict("fr_FR")
    #print "FR LOADED"
    lg_fr=0
    lg_ang=0
    #print "string_to_read",string_to_read
    for word in string_to_read.split():
        #print fr.check(word)
        #print word
        if fr.check(word) == True:
            lg_fr+=1
        if us.check(word) == True:
            lg_ang+=1
    #print "THERE I AM"
    if lg_fr >= lg_ang :
        return "french"
    else:
        if lg_ang > lg_fr :
        
            return "english"
        else: 
            return "NEITHER ENGLISH NOR FRENCH"
    
    

In [ ]:
def topic_analysis_with_mallet(string_to_read):
    lng = language_detection_with_pyenchant(string_to_read)
    if lng== "french" :
        lng= ["a","à","â","abord","afin","ah","ai","aie","ainsi","allaient","allo","allô","allons","après","assez","attendu","au","aucun","aucune","aujourd","aujourd'hui","auquel","aura","auront","aussi","autre","autres","aux","auxquelles","auxquels","avaient","avais","avait","avant","avec","avoir","ayant","b","bah","beaucoup","bien","bigre","boum","bravo","brrr","c","ça","car","ce","ceci","cela","celle","celle-ci","celle-là","celles","celles-ci","celles-là","celui","celui-ci","celui-là","cent","cependant","certain","certaine","certaines","certains","certes","ces","cet","cette","ceux","ceux-ci","ceuxlà","chacun","chaque","cher","chère","chères","chers","chez","chiche","chut","ci","cinq","cinquantaine","cinquante","cinquantième","cinquième","clac","clic","combien","comme","comment","compris","concernant","contre","couic","crac","d","da","dans","de","debout","dedans","dehors","delà","depuis","derrière","des","dès","désormais","desquelles","desquels","dessous","dessus","deux","deuxième","deuxièmement","devant","devers","devra","différent","différente","différentes","différents","dire","divers","diverse","diverses","dix","dix-huit","dixième","dix-neuf","dixsept","doit","doivent","donc","dont","douze","douzième","dring","du","duquel","durant","e","effet","eh","elle","elle-même","elles","ellesmêmes","en","encore","entre","envers","environ","es","ès","est","et","etant","étaient","étais","était","étant","etc","été","etre","être","eu","euh","eux","eux-mêmes","excepté","f","façon","fais","faisaient","faisant","fait","feront","fi","flac","floc","font","g","gens","h","ha","hé","hein","hélas","hem","hep","hi","ho","holà","hop","hormis","hors","hou","houp","hue","hui","huit","huitième","hum","hurrah","i","il","ils","importe","j","je","jusqu","jusque","k","l","la","là","laquelle","las","le","lequel","les","lès","lesquelles","lesquels","leur","leurs","longtemps","lorsque","lui","lui-même","m","ma","maint","mais","malgré","me","même","mêmes","merci","mes","mien","mienne","miennes","miens","mille","mince","moi","moi-même","moins","mon","moyennant","n","na","ne","néanmoins","neuf","neuvième","ni","nombreuses","nombreux","non","nos","notre","nôtre","nôtres","nous","nous-mêmes","nul","o","o|","ô","oh","ohé","olé","ollé","on","ont","onze","onzième","ore","ou","où","ouf","ouias","oust","ouste","outre","p","paf","pan","par","parmi","partant","particulier","particulière","particulièrement","pas","passé","pendant","personne","peu","peut","peuvent","peux","pff","pfft","pfut","pif","plein","plouf","plus","plusieurs","plutôt","pouah","pour","pourquoi","premier","première","premièrement","près","proche","psitt","puisque","q","qu","quand","quant","quanta","quant-à-soi","quarante","quatorze","quatre","quatre- vingt","quatrième","quatrièmement","que","quel","quelconque","quelle","quelles","quelque","quelques","quelqu'un","quels","qui","quiconque","quinze","quoi","quoique","r","revoici","revoilà","rien","s","sa","sacrebleu","sans","sapristi","sauf","se","seize","selon","sept","septième","sera","seront","ses","si","sien","sienne","siennes","siens","sinon","six","sixième","soi","soi-même","soit","soixante","son","sont","sous","stop","suis","suivant","sur","surtout","t","ta","tac","tant","te","té","tel","telle","tellement","telles","tels","tenant","tes","tic","tien","tienne","tiennes","tiens","toc","toi","toi-même","ton","touchant","toujours","tous","tout","toute","toutes","treize","trente","très","trois","troisième","troisièmement","trop","tsoin","tsouin","tu","u","un","une","unes","uns","v","va","vais","vas","vé","vers","via","vif","vifs","vingt","vivat","vive","vives","vlan","voici","voilà","vont","vos","votre","vôtre","vôtres","vous","vous-mêmes","vu","w","x","y","z","zut"]
    
    vectorizer = text.CountVectorizer(input=string_to_read, stop_words=lng, min_df=3)
    dtm = vectorizer.fit_transform(string_to_read.split()).toarray()
    vocab = np.array(vectorizer.get_feature_names())
    dtm.shape
    len(vocab)
    num_topics = 20
    num_top_words = 20
    clf = decomposition.NMF(n_components=num_topics, random_state=1)
    doctopic = clf.fit_transform(dtm)
    topic_words = []
    for topic in clf.components_:
        word_idx = np.argsort(topic)[::-1][0:num_top_words]
        topic_words.append([vocab[i] for i in word_idx])
    doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)
    return [topic_words,doctopic]

In [ ]:
#so far french support
def find_geo_topics(string_to_read, lng):
    geo=0
    geo_words=[]
    #print string_to_read
    if lng== "french":
        for word in (string_to_read.split()):
        #print word
        #LUSSAULT ;) https://www.espacestemps.net/articles/entrees-par-categories/
            # if (word in ["Théorie de l’espace","Accessibilité","Acteur spatial","Action spatiale","Agencement","Agglomération","Ailleurs","Aire","Aire culturelle","Alignement","Ambiance architecturale et urbaine","Aménagement du territoire","Anamorphose","Anthropisation","Archipel mégalopolitain mondial ","Armature urbaine","Attraction","Autocorrélation spatiale","Banlieue","Campagne","Capital spatial","Carte","Carte mentale","Centralité","Centre/Périphérie","Centre urbain","Chorème","Chorotype","Circulation","Citadinité","Communication territoriale","Commutateur","Compromis territorial","Concentration","Configuration spatiale","Confins","Connexité","Contact","Contiguïté","Continent","Continuité","Coprésence","Corps","Cospatialité","Cyberespace","Décentralisation","Découpage","Découverte","Défrichement","Densité","Désert","Déterritorialisation","Développement local","Diaspora","Différenciation spatiale -Diffusion","Discontinuité","Dispositif spatial légitime","Distance","Distribution rang/taille","Distribution spatiale","District industriel","Diversité","Dynamique spatiale","Écart","Échelle","Économie-monde","Écoumène","Edge City","Emblème territorial","Emboîtement","Empire","Enclavement -Ensemble géographique","Espace","Espace public ","Espace vécu","État","État local","Étendue","Fédéralisme","Finage","Firme transnationale","Fleuve","Flux","Foncier","Forêt","Fractale","Friche","Front","Front pionnier","Frontière","Générique ","Gentrification","Géoéconomie","Géogramme","Géographicité","Géographie","Géon","Géopolitique","Géosystème","Géotype","Ghetto","Glacis","Gouvernement urbain","Gradient","Graphe","Graphique","Gravitaire ","Guerre","Habitat","Habitat non-réglementaire","Habiter","Haut lieu","Heimat","Hétérotopie","Hinterland","Horizont","Hors-sol","Hub","Identité spatiale","Île","Image","Imaginaire géographique","Immanence/Transcendance ","Infra-urbain","Interaction spatiale","Interface","Interspatialité","Irrigation","Isolat","Isotropie","Jardin","Justice spatiale","Lieu","Lieux centraux ","Limite","Littoral","Local","Localisation","Logistique","Maillage","Maison","Marchandise","Médiance","Méditerranée","Mer","Métaphore spatiale","Métrique","Métropole/Mégalopole","Métropolisation","Migration","Milieu","Milieu innovateur","Minorité territoriale","Mobilité","Monde","Mondialisation","Montagne","Nation","Network","Nœud","Norme","Oasis","Objet géographique","Parc à thème","Parc naturel","Parcours","Partie du monde","Pavillonnaire ","Pays","Paysage","Périurbain","Peuplement","Polarisation","Population ","Position","Pratique spatiale","Projet urbain","Prospective territoriale","Proxémie","Reconstruction","Reconversion","Rénovation/Restauration/Réhabilitation","Représentation de l’espace","Réseau","Réseau technique","Réseau urbain","Rhizome","Rue","Rural","Schéma d’aménagement","Ségrégation","Seuil","Site","Situation géographique","Société-Monde","Sol","Spatialité","Stratégie spatiale","Substance","Système d’Information Géographique ","Système productif local ","Système spatial","Technopôle/Technopole","Télé-communication","Télétravail","Terre","Territoire","Territorial ","Territorialité","Terroir","Topogenèse","Topographie","Topologie","Toponymie","Tourisme","Transition démographique","Transports","Ubiquité","Urbain","Urbain ","Urbanisation","Urbanité","Valeur spatiale","Vaterland","Végétation","Village","Ville","Ville mondiale","Ville nouvelle","Violence","Visibilité ","Voisinage","Zonage","Zone climatique"]) or  "géogr" in word or "géomat" in word :
            if  "géogr" in word or "géomat" in word or "spatial" in word or "urbanis" in word:  #or "carto" in word:  
                geo+=1
                geo_words.append(word)
    if lng== "english":
        for word in (string_to_read.split()):
        #print word
        #LUSSAULT ;) https://www.espacestemps.net/articles/entrees-par-categories/
             if  "geogr" in word or "geomat" in word or "spatial" in word or "urbanis" in word: # or "carto" in word:
                geo+=1
                geo_words.append(word)
    print "SUJETS\n",geo_words,"\n points : ",geo
    return [geo, geo_words]

    

In [ ]:
def topic_analysis_with_manual_detection(string_to_read):
    lng = language_detection_with_pyenchant(string_to_read)
    if lng== "french" :
        geopoints= find_geo_topics(string_to_read, "french")
        #print string_to_read
    else:
        if lng== "english" :
            geopoints= find_geo_topics(string_to_read, "english")
        
        else:
            print "LANGUAGE NOT YET SUPPORTED"
            geopoints=['',''] 
    #    lng= ["a","à","â","abord","afin","ah","ai","aie","ainsi","allaient","allo","allô","allons","après","assez","attendu","au","aucun","aucune","aujourd","aujourd'hui","auquel","aura","auront","aussi","autre","autres","aux","auxquelles","auxquels","avaient","avais","avait","avant","avec","avoir","ayant","b","bah","beaucoup","bien","bigre","boum","bravo","brrr","c","ça","car","ce","ceci","cela","celle","celle-ci","celle-là","celles","celles-ci","celles-là","celui","celui-ci","celui-là","cent","cependant","certain","certaine","certaines","certains","certes","ces","cet","cette","ceux","ceux-ci","ceuxlà","chacun","chaque","cher","chère","chères","chers","chez","chiche","chut","ci","cinq","cinquantaine","cinquante","cinquantième","cinquième","clac","clic","combien","comme","comment","compris","concernant","contre","couic","crac","d","da","dans","de","debout","dedans","dehors","delà","depuis","derrière","des","dès","désormais","desquelles","desquels","dessous","dessus","deux","deuxième","deuxièmement","devant","devers","devra","différent","différente","différentes","différents","dire","divers","diverse","diverses","dix","dix-huit","dixième","dix-neuf","dixsept","doit","doivent","donc","dont","douze","douzième","dring","du","duquel","durant","e","effet","eh","elle","elle-même","elles","ellesmêmes","en","encore","entre","envers","environ","es","ès","est","et","etant","étaient","étais","était","étant","etc","été","etre","être","eu","euh","eux","eux-mêmes","excepté","f","façon","fais","faisaient","faisant","fait","feront","fi","flac","floc","font","g","gens","h","ha","hé","hein","hélas","hem","hep","hi","ho","holà","hop","hormis","hors","hou","houp","hue","hui","huit","huitième","hum","hurrah","i","il","ils","importe","j","je","jusqu","jusque","k","l","la","là","laquelle","las","le","lequel","les","lès","lesquelles","lesquels","leur","leurs","longtemps","lorsque","lui","lui-même","m","ma","maint","mais","malgré","me","même","mêmes","merci","mes","mien","mienne","miennes","miens","mille","mince","moi","moi-même","moins","mon","moyennant","n","na","ne","néanmoins","neuf","neuvième","ni","nombreuses","nombreux","non","nos","notre","nôtre","nôtres","nous","nous-mêmes","nul","o","o|","ô","oh","ohé","olé","ollé","on","ont","onze","onzième","ore","ou","où","ouf","ouias","oust","ouste","outre","p","paf","pan","par","parmi","partant","particulier","particulière","particulièrement","pas","passé","pendant","personne","peu","peut","peuvent","peux","pff","pfft","pfut","pif","plein","plouf","plus","plusieurs","plutôt","pouah","pour","pourquoi","premier","première","premièrement","près","proche","psitt","puisque","q","qu","quand","quant","quanta","quant-à-soi","quarante","quatorze","quatre","quatre- vingt","quatrième","quatrièmement","que","quel","quelconque","quelle","quelles","quelque","quelques","quelqu'un","quels","qui","quiconque","quinze","quoi","quoique","r","revoici","revoilà","rien","s","sa","sacrebleu","sans","sapristi","sauf","se","seize","selon","sept","septième","sera","seront","ses","si","sien","sienne","siennes","siens","sinon","six","sixième","soi","soi-même","soit","soixante","son","sont","sous","stop","suis","suivant","sur","surtout","t","ta","tac","tant","te","té","tel","telle","tellement","telles","tels","tenant","tes","tic","tien","tienne","tiennes","tiens","toc","toi","toi-même","ton","touchant","toujours","tous","tout","toute","toutes","treize","trente","très","trois","troisième","troisièmement","trop","tsoin","tsouin","tu","u","un","une","unes","uns","v","va","vais","vas","vé","vers","via","vif","vifs","vingt","vivat","vive","vives","vlan","voici","voilà","vont","vos","votre","vôtre","vôtres","vous","vous-mêmes","vu","w","x","y","z","zut"]
    
    return geopoints

In [52]:
def read_html_pages_in_dirs_and_extract_mails_dict(pages_dir):
    dict_mails={}
    #rootdir="./DHFR_sample/"
    count_Cc=0
    count_message=0
    for subdir, dirs, files in os.walk(pages_dir):
#ON VA LIRE TOUS LES FICHIERS DU REP ROOTDIR
        
        for file in files:
    #V1     if file.endswith('.html') == True  and file != ("index.html") and not re.match("mail\d+.html",file) : 
           if re.match("msg\d+.html",file) :
          
            # les index.html sont des recap, on pourrait les compter pour s'assurer du nombre de messages
                
                #!print "\n",os.path.join(subdir, file), "\n"
                champs_mail={}
                filename=os.path.join(subdir, file)
                champs_mail["Geo_Topic"]=read_html_and_testtextextract(filename)
                
                
                soup=BeautifulSoup(open(filename, 'r').read(),'lxml')#, 'html.parser') 
#RECUPERER LE SUJET DU MESSAGE
                
                i=0
                for zone_cible in soup.findAll('ul'):
                    i+=1
                    sujet_messg = zone_cible.find(string=re.compile("\[DH\]"))
                    zone_de_metadonnees = None
                    special_auteur=0
##ON ISOLE LE CHAMPS AUTEUR
                    if '<li><strong>From</strong>:' in str(zone_cible) and '<li><strong>To</strong>:' in str(zone_cible) :
                        #print "FROM/TO DETECTED, i= ",i
                        count_message+=1
                        #print  zone_cible
                        nom_auteur= re.sub(r'<ul>\n<li><strong>From</strong>: ', '',str(zone_cible))
                        sep = "&lt;"
                        nom_auteur = nom_auteur.split(sep, 1)[0]
                        
                        
                        mail1= re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n document\.write\(\"', '',str(zone_cible))
                        mail2=re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n document\.write\(\".+@\" \+ \"', '',str(zone_cible))
                        sep1 = "\" + \"@\" + "
                        sep2 = "\")"
                        mail1= mail1.split(sep1, 1)[0]
                        mail2= mail2.split(sep2, 1)[0]
                        mail_auteur= mail1+"@"+mail2
                        #print mail_auteur
                        if len(mail_auteur)>60:
                            try:
                                mail= re.sub(r'<ul>\n<li><strong>From</strong>:', '',str(zone_cible))
                                mail=re.search('&lt;(.*)&gt;',str(mail))
                                mail=mail.group(1)
                                #print "MAILBIS",mail
                                mail_auteur=mail
                                #special_auteur=1
                            except:
                                print "BUG"
                        #CHECK PROPER CAPTURE OF THE NAME    
                        if len(nom_auteur)>50:
                            #print zone_cible
                            nom_auteur=mail_auteur
                        if len(nom_auteur)<5:
                            nom_auteur=mail_auteur
                        if nom_auteur[0]== "\"":
                            nom_auteur=nom_auteur[1:]
                            nom_auteur=nom_auteur[:-2]
                            print nom_auteur
                        if nom_auteur.endswith(' '):
                            nom_auteur=nom_auteur[:-1]
                        #!print "nom_auteur ",nom_auteur+" mail_auteur "+mail_auteur,"\n"
                        champs_mail["nom_auteur"]=  nom_auteur
                        champs_mail["mail_auteur"]=  mail_auteur
                        champs_mail["ref_physique_de_l_article"]=  os.path.join(subdir, file)
                        champs_mail["sujet_du_message"] = sujet_messg
##ON ISOLE LE CHAMP DESTINATAIRES
                        #if special_auteur == 1:
                            #print "ZCspecAut",zone_cible
                        #else: 
                        
                        destinataires=re.sub(r'<ul>\n<li><strong>From</strong>:.+\n.+\n.+\n.+\n.+\n.+\n.+\n', '',str(zone_cible))
                        
                        sep3=";</li>"
                        destinataires= destinataires.split(sep3, 1)[0]
                        #print destinataires
                        dest=re.findall("<script type=.*\n.+\n.+\n.+\n.+/script>", destinataires)
                        #print dest
                        liste_destinataires=[]
                        for destinataire in dest:
                            #print destinataire,"\n___________________\n"
                            mail1= re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\"', '',str(destinataire))
                            mail2=re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\".+@\" \+ \"', '',str(destinataire))
                            sep1 = "\" + \"@\" + "
                            sep2 = "\")"
                            mail1= mail1.split(sep1, 1)[0]
                            mail2= mail2.split(sep2, 1)[0]
                            mail_destinataire= mail1+"@"+mail2
                            #print mail_destinataire,"\n___________\n"
                            if mail_destinataire not in liste_destinataires :
                                liste_destinataires.append(mail_destinataire)
                        if liste_destinataires==[]:
                            #print zone_cible
                            liste_destinataires=['dh@groupes.renater.fr']
                        #!print "destinataires: ",liste_destinataires,"\n"
                        champs_mail["liste_mails_dests"] = liste_destinataires
                       
                        if '<li><strong>Cc</strong>:' in str(zone_cible):
##ON ISOLE LE CHAMP CC
                            #print "CC DETECTED, i= ",i
                            #print zone_cible
                            count_Cc+=1
                            CCfield= re.search('<li><strong>Cc</strong>:(.*)<li><strong>Subject</strong>',str(zone_cible),flags=re.DOTALL)
                            CC= CCfield.group(1)
                            CC=str(CC)
                            CC=re.sub(r'^.+\n','',str(CC),flags=0)
                            CCs=re.findall("<script type=.*\n.+\n.+\n.+\n.+/script>", CC)
                            #sep1 = "\" + \"@\" + "
                            #mail1= mail1.split(sep1, 1)[0]
                            #print "CC FIELd\n"#,CCs
                            liste_CCs=[]
                            for CC in CCs:
                                #print CC,"\n___________________\n"
                                mail1= re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\"', '',str(CC))
                                mail2=re.sub(r'<script type=\"text/javascript\">\n <!-- \n document\.write\(\".+@\" \+ \"', '',str(CC))
                                sep1 = "\" + \"@\" + "
                                sep2 = "\")"
                                mail1= mail1.split(sep1, 1)[0]
                                mail2= mail2.split(sep2, 1)[0]
                                mail_CC= mail1+"@"+mail2
                                #print mail_CC,"\n___________\n"
                                if mail_CC not in liste_CCs :
                                    liste_CCs.append(mail_CC)
                            #!print "CCs: ",liste_CCs
                            champs_mail["liste_CCs"] = liste_CCs
##ON ISOLE LE CHAMP DATE                       
                        #print "ZONECIBLE",zone_cible
                        champs_datef=re.search('<li><strong>Date</strong>:(.+?)</li>',str(zone_cible),flags=re.DOTALL)
                        #print "CHAMPSDATEF",champs_datef
                        try :
                            champs_date=champs_datef.group(1)
                        except :
                            #print "ZCparent", zone_cible.parent
                            champs_date=re.search('<!--X-Date: (.*) -->',str(soup))
                            champs_date=champs_date.group(1)
                            #print champs_date
                            
                        #print champs_date
                        try : 
                            champs_date=parser.parse(champs_date)
                        except ValueError :
                            sep=" ("
                            champs_date=champs_date.split(sep, 1)[0]
                            
                        #!print  champs_date
                        champs_mail["date"] = champs_date
                        
                        
                        dict_mails[os.path.join(subdir, file)]= champs_mail
                        break
##LE DICTIONNAIRE DES MAILS SE NOMME dict_mails
    #print dict_mails
    print count_message," messages"
    print count_Cc, " messages avec Cc"
    ratiocount = float(count_Cc)/float(count_message)
    print "Taux_de_mails_avec_CC:",ratiocount,"\n"
    return dict_mails

In [ ]:
def read_corpus_and_extract_topo_network(corpus):
    return

In [ ]:
def read_corpus_and_extract_topo_Geo_network(corpus):
    return

In [ ]:
def read_corpus_and_extract_mails(corpus):
    liste_mails=[]
    for mesg in corpus:
        #print corpus[mesg]
        if corpus[mesg]['mail_auteur'] not in liste_mails:
            liste_mails.append(corpus[mesg]['mail_auteur'])
        for i,key in enumerate(corpus[mesg]['liste_mails_dests']):
            #print key
            if key not in liste_mails:
                liste_mails.append(key)
        if 'liste_CCs' in corpus[mesg] :
            #print "TRUVE CC"
            for i,key in enumerate(corpus[mesg]['liste_CCs']):
                #print key
                if key not in liste_mails:
                    liste_mails.append(key)
    return liste_mails

In [ ]:
def read_corpus_and_extract_localisations(corpus):
    return

In [ ]:
def read_corpus_and_extract_names(corpus):
    liste_noms=[]
    for mesg in corpus:
        try: 
            if corpus[mesg]['nom_auteur'][0]== "\"":
                print corpus[mesg]['nom_auteur']
        except IndexError:
            print "Bug", corpus[mesg]['nom_auteur']
            print corpus[mesg]
        if [corpus[mesg]['nom_auteur'],corpus[mesg]['mail_auteur']] not in liste_noms:
            for i,key in enumerate(liste_noms):
                if SequenceMatcher(None, corpus[mesg]['nom_auteur'], key[0]).ratio() >=0.9:
                    print "POSSIBLE DUPLICATE |", corpus[mesg]['nom_auteur'],"|   |",key[0],"|"
                    print corpus[mesg]['mail_auteur'], key[1]
                
            liste_noms.append([corpus[mesg]['nom_auteur'],corpus[mesg]['mail_auteur']])
        
    return liste_noms

In [59]:
corpus_mails_DH=read_html_pages_in_dirs_and_extract_mails_dict("./DHFRsample")
#print corpus_mails_DH



SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
Noiret, Serge
SUJETS
[] 
 points :  0
Noiret, Serge
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
['g\xc3\xa9ographie,'] 
 points :  1
SUJETS
['g\xc3\xa9ographie,'] 
 points :  1
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
[] 
 points :  0
SUJETS
['g\xc3\xa9ographie,'] 
 points :  1
SUJETS
[] 
 points :  0
Ghislain SILLAUME
SUJETS
['g\xc3\xa9ographie,'] 
 points :  1
SUJETS
['g\xc3\xa9ographie,'] 
 points :  1
SUJETS
['g\xc3\xa9ographie,'] 
 points :  1
SUJETS
[] 
 points :  0
SUJETS
['g\xc3\xa9ographie,'] 
 points :  1
SUJETS
[] 
 points :  0
SUJETS
['g\xc3\xa9ographie,'] 
 points :  1
SUJETS
[] 
 points :  0
SU

In [ ]:
liste_mails=read_corpus_and_extract_mails(corpus_mails_DH)
print len(liste_mails)
print liste_mails


In [ ]:
liste_noms=read_corpus_and_extract_names(corpus_mails_DH)
print len(liste_noms)
print liste_noms

In [ ]:
def read_html_pages_in_dirs_and_testtextextract(pages_dir):
    dict_mails={}
    #rootdir="./DHFR_sample/"

    
    for subdir, dirs, files in os.walk(pages_dir):
#ON VA LIRE TOUS LES FICHIERS DU REP ROOTDIR
        
        for file in files:
            dictT={}
    #V1     if file.endswith('.html') == True  and file != ("index.html") and not re.match("mail\d+.html",file) : 
            if re.match("msg\d+.html",file):
                filename=os.path.join(subdir, file)
                with open(filename, 'r') as myfile:           
                
                    data=myfile.read()
                    result = re.search('<!--X-Body-of-Message-->(.*)<!--X-Body-of-Message-End-->', data, flags=re.DOTALL)
                    ZC= result.group(1)
                    ZC=cleanhtml(ZC)
                    dictT["content"]=ZC
                    dictT["geo"]=topic_analysis_with_manual_detection(ZC)
                dict_mails[filename]= dictT
            
    return dict_mails

In [58]:
def read_html_and_testtextextract(filename):
                dictT={}
                with open(filename, 'r') as myfile:
                    data=myfile.read()
                    result = re.search('<!--X-Body-of-Message-->(.*)<!--X-Body-of-Message-End-->', data, flags=re.DOTALL)
                    ZC= result.group(1)
                    ZC=cleanhtml(ZC)
                    dictT["content"]=ZC
                    dictT["geo"]=topic_analysis_with_manual_detection(ZC)
                return dictT

In [ ]:
corpus_txt=read_html_pages_in_dirs_and_testtextextract("./DHFRsample")
#print corpus_txt
#lng= language_detection_with_pyenchant(str(corpus_txt))
#print lng

In [ ]:
import enchant
print enchant.list_languages()

In [ ]:
Analyse =topic_analysis_with_nltk_gensim(corpus_txt)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
print(Analyse)

In [ ]:
def topic_analysis_with_nltk_gensim(string_to_read):
    string_to_read=unicode(str(string_to_read), 'utf8')
    lng = language_detection_with_pyenchant(string_to_read)
   
    from nltk.corpus import stopwords
    from nltk.stem.wordnet import WordNetLemmatizer
    import string
    print lng
    stop = set(stopwords.words(lng))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized

    doc_clean = clean(string_to_read).split()
    print "DOC CLEAN",doc_clean
        # Importing Gensim
    import gensim
    from gensim import corpora

        # Creating the term dictionary of our courpus, where every unique term is assigned an index. 
    dictionary = corpora.Dictionary([doc_clean])

        # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = dictionary.doc2bow(doc_clean)
    Lda = gensim.models.ldamodel.LdaModel
    print Lda

        # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=5)
    return ldamodel
       
        
    
    
    
    #return [topic_words,doctopic]